# COVID19 - District Region of Sao Paulo State, Brazil

In [1]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.str("CUDA_VISIBLE_DEVICES",'0,1,2,3,4')
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
# MB=1024*1024
# GB=MB*1024
# ray.init(num_gpus=5,num_cpus=1)
# @ray.remote(num_gpus=1)
# def use_gpu():
#     print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
#     print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

# use_gpu.remote()


ray.shutdown()
ray.init()

2021-03-02 09:56:27,256	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8267


{'node_ip_address': '192.168.0.104',
 'raylet_ip_address': '192.168.0.104',
 'redis_address': '192.168.0.104:42691',
 'object_store_address': '/tmp/ray/session_2021-03-02_09-56-25_729378_7546/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-03-02_09-56-25_729378_7546/sockets/raylet',
 'webui_url': '127.0.0.1:8267',
 'session_dir': '/tmp/ray/session_2021-03-02_09-56-25_729378_7546',
 'metrics_export_port': 64263,
 'node_id': '7469b8b97d3f38dfaea35d022d6ccd00602f1dc5'}

In [2]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date

In [3]:
# Download data
import get_data
LoadData=False

if LoadData:
    get_data.get_data()

In [4]:
dfSP = pd.read_csv("data/dados_municipios_SP.csv.gz",compression='gzip')
query=dfSP.query('DRS == "{}"'.format('DRS 01 - Grande São Paulo'))
query2=query[query['date']=="2020-12-27"]
display(query2)
display(query2['popEst'].sum())

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,DRS
11007,2020-12-27,SP,Arujá,city,2812,111,262,False,89824.0,91157.0,3503901.0,3084.78778,0.0395,DRS 01 - Grande São Paulo
16271,2020-12-27,SP,Barueri,city,11200,440,266,False,274182.0,276982.0,3505708.0,4043.58406,0.0393,DRS 01 - Grande São Paulo
19512,2020-12-27,SP,Biritiba Mirim,city,557,33,236,False,32598.0,32936.0,3506607.0,1691.15861,0.0592,DRS 01 - Grande São Paulo
27145,2020-12-27,SP,Caieiras,city,3258,109,263,False,101470.0,102775.0,3509007.0,3170.03162,0.0335,DRS 01 - Grande São Paulo
27722,2020-12-27,SP,Cajamar,city,1850,84,260,False,76801.0,77934.0,3509205.0,2373.80347,0.0454,DRS 01 - Grande São Paulo
32394,2020-12-27,SP,Carapicuíba,city,11966,435,268,False,400927.0,403183.0,3510609.0,2967.88307,0.0364,DRS 01 - Grande São Paulo
39241,2020-12-27,SP,Cotia,city,5679,230,266,False,249210.0,253608.0,3513009.0,2239.28267,0.0405,DRS 01 - Grande São Paulo
42309,2020-12-27,SP,Diadema,city,11367,479,261,False,423884.0,426757.0,3513801.0,2663.57670,0.0421,DRS 01 - Grande São Paulo
46575,2020-12-27,SP,Embu das Artes,city,4485,208,263,False,273726.0,276535.0,3515004.0,1621.85618,0.0464,DRS 01 - Grande São Paulo
46898,2020-12-27,SP,Embu-Guaçu,city,1710,58,253,False,69385.0,69901.0,3515103.0,2446.31693,0.0339,DRS 01 - Grande São Paulo


21734682.0

# Model

In [5]:
# lista DRSs
DRS = list(dfSP["DRS"].unique())
DRS.remove("Indefinido")
DRS

['DRS 09 - Marília',
 'DRS 15 - São José do Rio Preto',
 'DRS 14 - São João da Boa Vista',
 'DRS 06 - Bauru',
 'DRS 16 - Sorocaba',
 'DRS 11 - Presidente Prudente',
 'DRS 05 - Barretos',
 'DRS 13 - Ribeirão Preto',
 'DRS 02 - Araçatuba',
 'DRS 07 - Campinas',
 'DRS 03 - Araraquara',
 'DRS 10 - Piracicaba',
 'DRS 17 - Taubaté',
 'DRS 08 - Franca',
 'DRS 01 - Grande São Paulo',
 'DRS 12 - Registro',
 'DRS 04 - Baixada Santista']

# SEAIR-D Model Equations

$$\Large \begin{array}{l}
\dfrac{d s}{d t}=-[ i(t) + \beta_R \cdot a(t)] \cdot \beta \cdot s(t) \\ 
\dfrac{d e}{d t}=[i(t) + \beta_R \cdot a(t)] \cdot \beta \cdot s(t) -\sigma \cdot e(t) - \mu \cdot e(t)\\ 
\dfrac{d a}{d t}=\sigma \cdot e(t) \cdot (1-p)-\gamma \cdot a(t)  \\
\dfrac{d i}{d t}=\sigma \cdot e(t) \cdot p - \gamma \cdot i(t)  - \mu \cdot i(t)\\ 
\dfrac{d r}{d t}=b_\gamma \cdot i(t) + \gamma \cdot a(t) + \nu \cdot k(t)- \mu \cdot r(t)\\
\dfrac{d k}{d t}=(a_\gamma + \nu ) \cdot k(t) + \mu \cdot [e(t)+i(t)+r(t)]
\end{array}$$

The recovery and death rates are calculated from $\gamma$

$$\Large \gamma=a_\gamma+b_\gamma$$

The sigmoid function:

$$\Large S(t)=\frac{1}{1 + e^{-t}}$$

The $\sigma$ variation is modeled as

$$\Large 
    \sigma = \sigma_i \cdot [1-S(t)]+\sigma_f \cdot S(t)
$$

See below the transition between two value - $\sigma_i$ and $\sigma_f$ -  based on Sigmoids Curves when $\sigma_i$<$\sigma_f$, which is indicated by the blue curve, and $\sigma_i$>$\sigma_f$, which is indicated by the red curve.

![Sigmoids Curves when $\sigma_i$<$\sigma_f$ and $\sigma_i$>$\sigma_f$](../countries/results/sigmoid.png)
<h4><center>Sigmoids Curves when $\sigma_i$<$\sigma_f$ and $\sigma_i$>$\sigma_f$</center></h4>


The $\sigma$ with sigmoid is performed two times inside the data interval, so we have an initial value constant of $\sigma_i$, a sigmoid transition to value $\sigma_f$, a period of $\sigma_f$ constant, after a sigmoid transition to value $\sigma_{ff}$ and a period of $\sigma_{ff}$ constant. The times where the two sigmoids are placed are calculated by optimization as well as the values of $\sigma_i$,$\sigma_f$ and $\sigma_{ff}$.

The sum of all people accounted is equal to population considered:

$$\Large s + e + a + i + r + k = N$$

The derivate is:

$$\Large \frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t}+\frac{d k}{d t} = 0$$

The last equation does not need to be solve because:

$$\Large \frac{d k}{d t}=-(\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

### Fixed parameters:

N: the population considered in the study

$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

### Optimized Parameters

All parameters below are calculated through evolutionary optimization package Yabox to fit real data:

$\beta$: Effective contact rate [1/day]

$\beta_R$: Fraction of effective contact rate [1/day]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a_\gamma+b_\gamma)$ [1/day]

$a_\gamma$: mortality of infected  [1/day]

$b_\gamma$: recovery rate  [1/day]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Three parameters $\sigma_i$, $\sigma_f$ and $\sigma_{ff}$ are used along the days. The transition from $\sigma_i$ to $\sigma_f$ and from $\sigma_f$ to $\sigma_{ff}$ are made by a sigmoid function. Its reciprocal  ($1/\sigma$) is the average latent (exposed) period. [1/day]

$\nu$: medicine and hospital improvement rate [1/day] 

$\mu$: direct mortality rate [1/day] 

## Options: Param.CSV file, changeCSV, Adjust Param, Save CSV, Select Specific DRS

In [6]:
from IPython.display import display, HTML, Markdown

paramOpt=2
changeCSV=False
adjustParam=False
paramSave=False
selectDRS=False

## Different Options to Load Param.CSV file

In [7]:
if paramOpt==-1:
    paramFile="data/param.csv"
    version = "-1"
    model = "ManualIC"
    
if paramOpt==0:
    paramFile= "data/param_optimized_Yabox_HistMin.csv"
    version = "0"
    model = "LastIC"
    
if paramOpt==1:
    version = "116"
    model = "YaboxIC"    
    paramFile="data/param_optimized_Yabox_HistMin"+version+".csv"

if paramOpt==2:
    version = "119"
    model = "YaboxIC"    
    paramFile="data/param_optimized_Yabox_HistMin"+version+".csv"


In [8]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam.e0=0
dfparam.r0=0

display(Markdown("## Original Initial Parameters"))
display(dfparam)

## Original Initial Parameters

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-17,200,12401753,0,0,77,0,144,28,0.15,0.6916,0.0
1,DRS 02 - Araçatuba,2020-04-03,200,764871,0,0,95,0,178,216,0.15,0.2348,0.0
2,DRS 03 - Araraquara,2020-03-31,200,559757,0,0,220,0,75,65,0.15,0.2097,0.0
3,DRS 04 - Baixada Santista,2020-03-29,200,1404254,0,0,128,0,162,68,0.15,0.2692,0.0
4,DRS 05 - Barretos,2020-04-01,200,292016,0,0,69,0,53,204,0.15,0.5739,0.0
5,DRS 06 - Bauru,2020-03-29,200,1579258,0,0,92,0,98,70,0.15,0.2023,0.0
6,DRS 07 - Campinas,2020-03-29,200,2296398,0,0,108,0,165,177,0.15,0.3124,0.0
7,DRS 08 - Franca,2020-04-04,200,269546,0,0,81,0,56,12,0.15,0.4050,0.0
8,DRS 09 - Marília,2020-04-01,200,563617,0,0,151,0,51,188,0.15,0.4492,0.0
9,DRS 10 - Piracicaba,2020-04-04,200,1186414,0,0,105,0,3,131,0.15,0.3074,0.0


## Modify param.csv file to fit better the data

In [9]:
#     dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
#     dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
#     dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
#     dfparam['s0'] = dfparam['s0'].astype(float)
#     dfparam['i0'] = dfparam['i0'].astype(float)
#     dfparam['d0'] = dfparam['d0'].astype(float)
#     dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','s0'] = dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','s0'].values[0]*1.1
#     dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','i0'] = dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','i0']*.1

if changeCSV and (paramOpt==2):
    dfparam.loc[dfparam.DRS=='DRS 05 - Barretos','s0'] = dfparam.loc[dfparam.DRS=='DRS 05 - Barretos','s0']*2
    dfparam.loc[dfparam.DRS=='DRS 05 - Barretos','d0'] = dfparam.loc[dfparam.DRS=='DRS 05 - Barretos','d0']/10
    dfparam.loc[dfparam.DRS=='DRS 12 - Registro','s0'] = dfparam.loc[dfparam.DRS=='DRS 12 - Registro','s0']*2
    dfparam.loc[dfparam.DRS=='DRS 12 - Registro','d0'] = dfparam.loc[dfparam.DRS=='DRS 12 - Registro','d0']/10
    dfparam.loc[dfparam.DRS=='DRS 08 - Franca','s0'] = dfparam.loc[dfparam.DRS=='DRS 08 - Franca','s0']*2
    dfparam.loc[dfparam.DRS=='DRS 08 - Franca','d0'] = dfparam.loc[dfparam.DRS=='DRS 08 - Franca','d0']/100    
    
if adjustParam:
    sCorrect=[1.38,1.0,1.05,1.5,1.5,1.4,1.1,1.5,1.15,1.55,1.35,1.2,1.35,1.15,1.55,1.45,1.25]
    dfparam.s0=dfparam.s0.multiply(sCorrect[:len(DRS)], axis=0)

if selectDRS:
    DRS=pd.Series(['DRS 08 - Franca','DRS 12 - Registro','DRS 05 - Barretos','DRS 01 - Grande São Paulo'])
else:
    DRS=dfparam.DRS    

dfparam['s0'] = dfparam['s0'].astype(int)
dfparam['i0'] = dfparam['i0'].astype(int)
dfparam['d0'] = dfparam['d0'].astype(int)
dfparam['r0'] = dfparam['r0'].astype(int) 
    
display(Markdown("## Modified Initial Parameters"))

# dfparam['WREC']=dfparam['WREC'].apply(lambda x: min(x,0.001))
dfparam['WDTH']=1-dfparam['WCASES']-dfparam['WREC']

finalDate = date.today() + timedelta(days=-1)
finalDateStr = datetime.strftime(finalDate, "%Y-%m-%d")
dfparam['endDate']=finalDateStr

#poe RATIO no final
l=[0,1,2,3,4,5,6,7,8,9,11,12, 13, 14, 10] # index order
dfparam=dfparam[[dfparam.columns[i] for i in l]]

# for i in DRS:
#     query=dfSP.query('DRS == "{}"'.format(i))
#     query2=query[query['date']=="2020-12-28"]
#     dfparam.loc[dfparam['DRS']==i,'s0']=int(query2['popEst'].sum()*0.7)
    
if paramSave:
    dfparam.to_csv("new_"+paramFile)       
    
display(dfparam)


## Modified Initial Parameters

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH,endDate,RATIO
0,DRS 01 - Grande São Paulo,2020-03-17,200,12401753,0,0,77,0,144,28,0.6916,0.0,0.3084,2021-03-01,0.15
1,DRS 02 - Araçatuba,2020-04-03,200,764871,0,0,95,0,178,216,0.2348,0.0,0.7652,2021-03-01,0.15
2,DRS 03 - Araraquara,2020-03-31,200,559757,0,0,220,0,75,65,0.2097,0.0,0.7903,2021-03-01,0.15
3,DRS 04 - Baixada Santista,2020-03-29,200,1404254,0,0,128,0,162,68,0.2692,0.0,0.7308,2021-03-01,0.15
4,DRS 05 - Barretos,2020-04-01,200,292016,0,0,69,0,53,204,0.5739,0.0,0.4261,2021-03-01,0.15
5,DRS 06 - Bauru,2020-03-29,200,1579258,0,0,92,0,98,70,0.2023,0.0,0.7977,2021-03-01,0.15
6,DRS 07 - Campinas,2020-03-29,200,2296398,0,0,108,0,165,177,0.3124,0.0,0.6876,2021-03-01,0.15
7,DRS 08 - Franca,2020-04-04,200,269546,0,0,81,0,56,12,0.4050,0.0,0.5950,2021-03-01,0.15
8,DRS 09 - Marília,2020-04-01,200,563617,0,0,151,0,51,188,0.4492,0.0,0.5508,2021-03-01,0.15
9,DRS 10 - Piracicaba,2020-04-04,200,1186414,0,0,105,0,3,131,0.3074,0.0,0.6926,2021-03-01,0.15


## Solver Loading

In [10]:
%reload_ext autoreload
%autoreload 2
import LearnerYabox_v5NewModel as Learner #Yabox

2021-03-02 09:56:43,307	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8267


## All Districts and Select District to Run

In [11]:
allDistricts=True
cleanRecovered=False

if allDistricts:
    display(DRS)
else:
    districtRegionSelected="DRS 08 - Franca"
    display(districtRegionSelected)

0          DRS 01 - Grande São Paulo
1                 DRS 02 - Araçatuba
2                DRS 03 - Araraquara
3          DRS 04 - Baixada Santista
4                  DRS 05 - Barretos
5                     DRS 06 - Bauru
6                  DRS 07 - Campinas
7                    DRS 08 - Franca
8                   DRS 09 - Marília
9                DRS 10 - Piracicaba
10      DRS 11 - Presidente Prudente
11                 DRS 12 - Registro
12           DRS 13 - Ribeirão Preto
13    DRS 14 - São João da Boa Vista
14    DRS 15 - São José do Rio Preto
15                 DRS 16 - Sorocaba
16                  DRS 17 - Taubaté
Name: DRS, dtype: object

In [12]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

# Main Loop

In [ ]:
if allDistricts:

    if selectDRS:
        loop=[0,len(DRS)]        
    else:
        loop=[0,6,12,17]
    
    for i in range(0,len(loop)-1):
        results=[]
        for districtRegion in DRS[loop[i]:loop[i+1]]:
            query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
            parameters = np.array(query.iloc[:, 2:])[0]
            dR=ray.put(districtRegion)
            cR=ray.put(cleanRecovered)
            vR=ray.put(version)
            f=Learner.Learner.remote(dR, *parameters, cR, vR)
            result = f.train.remote() 
            results.append(result)
        display("Running DRS {:d} to {:d}".format(loop[i]+1,loop[i+1]))
        display(DRS[loop[i]:loop[i+1]])
        resultsFinal = ray.get(results)

else:
    results=[]
    districtRegion= districtRegionSelected 
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    dR=ray.put(districtRegion)
    cR=ray.put(cleanRecovered)
    vR=ray.put(version)
    f=Learner.Learner.remote(dR, *parameters, cR, vR)
    result = f.train.remote() 
    results.append(result)
    display(districtRegion)
    resultsFinal = ray.get(results)


'Running DRS 1 to 6'

0    DRS 01 - Grande São Paulo
1           DRS 02 - Araçatuba
2          DRS 03 - Araraquara
3    DRS 04 - Baixada Santista
4            DRS 05 - Barretos
5               DRS 06 - Bauru
Name: DRS, dtype: object

  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:04<?, ?it/s]
  0%|          | 0/10125000.0 [00:04<?, ?it/s]
  0%|          | 0/10125000.0 [00:04<?, ?it/s]
  0%|          | 0/10125000.0 [00:04<?, ?it/s]
  0%|          | 0/10125000.0 [00:04<?, ?it/s]
  0%|          | 0/10125000.0 [00:04<?, ?it/s]
  0%|          | 1/10125000.0 [00:04<1176:39:46,  2.39it/s]
  0%|          | 1/10125000.0 [00:04<1330:52:49,  2.11it/s]
  0%|          | 1/10125000.0 [00:04<1288:17:49,  2.18it/s]
  0%|          | 1/10125000.0 [00:04<1353:23:48,  2.08it/s]
  0%|          | 1/10125000.0 [00:04<1322:26:39,  2.13it/s]
  0%|          | 3/10125000.0 [00:05<1006:53:59,  2.79it/s]
  0%|          | 1/10125000.0 [00:05<1564:34:18,  1.80it/s]
  0%|          |

  0%|          | 253/10125000.0 [00:15<79:01:09, 35.59it/s]
  0%|          | 276/10125000.0 [00:15<69:22:57, 40.53it/s]
  0%|          | 253/10125000.0 [00:15<79:33:31, 35.35it/s]
  0%|          | 351/10125000.0 [00:16<57:15:12, 49.12it/s]
  0%|          | 190/10125000.0 [00:16<110:32:29, 25.44it/s]
  0%|          | 276/10125000.0 [00:16<74:43:21, 37.64it/s]
  0%|          | 276/10125000.0 [00:16<75:25:18, 37.29it/s]
  0%|          | 300/10125000.0 [00:16<66:23:59, 42.36it/s]
  0%|          | 276/10125000.0 [00:16<75:39:13, 37.17it/s]
  0%|          | 378/10125000.0 [00:16<55:24:51, 50.75it/s]
  0%|          | 300/10125000.0 [00:16<71:25:41, 39.37it/s]
  0%|          | 325/10125000.0 [00:16<63:24:45, 44.35it/s]
  0%|          | 300/10125000.0 [00:16<72:16:01, 38.92it/s]
  0%|          | 210/10125000.0 [00:16<104:23:47, 26.94it/s]
  0%|          | 300/10125000.0 [00:16<72:07:55, 38.99it/s]
  0%|          | 406/10125000.0 [00:17<53:54:03, 52.18it/s]
  0%|          | 325/10125000.0 [00:17

  0%|          | 1035/10125000.0 [00:29<38:30:33, 73.03it/s]
  0%|          | 1035/10125000.0 [00:29<39:18:01, 71.56it/s]
  0%|          | 1128/10125000.0 [00:29<36:33:43, 76.92it/s]
  0%|          | 1378/10125000.0 [00:29<30:23:43, 92.52it/s]
  0%|          | 1081/10125000.0 [00:29<38:58:07, 72.17it/s]
  0%|          | 780/10125000.0 [00:29<51:49:13, 54.27it/s]
  0%|          | 1081/10125000.0 [00:29<37:36:35, 74.77it/s]
  0%|          | 1081/10125000.0 [00:29<37:56:16, 74.13it/s]
  0%|          | 1176/10125000.0 [00:29<35:22:33, 79.49it/s]
  0%|          | 1128/10125000.0 [00:30<37:54:45, 74.18it/s]
  0%|          | 1431/10125000.0 [00:29<30:37:01, 91.85it/s]
  0%|          | 820/10125000.0 [00:30<51:33:00, 54.55it/s]
  0%|          | 1225/10125000.0 [00:30<34:46:08, 80.88it/s]
  0%|          | 1128/10125000.0 [00:30<37:58:24, 74.06it/s]
  0%|          | 1128/10125000.0 [00:30<38:48:52, 72.45it/s]
  0%|          | 1485/10125000.0 [00:30<29:51:05, 94.20it/s]
  0%|          | 1176/1012

...

  3%|▎         | 286146/10125000.0 [09:38<2:17:34, 1191.90it/s]
  3%|▎         | 255970/10125000.0 [09:39<2:27:37, 1114.23it/s]
  3%|▎         | 288420/10125000.0 [09:38<2:11:02, 1251.03it/s]
  2%|▏         | 244650/10125000.0 [09:39<2:36:30, 1052.12it/s]
  2%|▏         | 220116/10125000.0 [09:39<2:55:40, 939.72it/s]
  3%|▎         | 300700/10125000.0 [09:38<2:08:44, 1271.76it/s]
  3%|▎         | 286903/10125000.0 [09:39<2:18:36, 1182.93it/s]
  3%|▎         | 289180/10125000.0 [09:39<2:09:39, 1264.26it/s]
  3%|▎         | 256686/10125000.0 [09:39<2:28:12, 1109.75it/s]
  2%|▏         | 245350/10125000.0 [09:39<2:35:58, 1055.66it/s]
  3%|▎         | 301476/10125000.0 [09:39<2:07:35, 1283.27it/s]
  2%|▏         | 220780/10125000.0 [09:39<2:56:33, 934.97it/s]
  3%|▎         | 287661/10125000.0 [09:39<2:19:26, 1175.79it/s]
  3%|▎         | 289941/10125000.0 [09:39<2:09:29, 1265.92it/s]
  3%|▎         | 257403/10125000.0 [09:40<2:27:32, 1114.67it/s]
  3%|▎         | 302253/10125000.0 [09:39<

  2%|▏         | 232903/10125000.0 [09:52<2:52:51, 953.74it/s]
  3%|▎         | 306153/10125000.0 [09:52<2:01:35, 1345.91it/s]
  3%|▎         | 318003/10125000.0 [09:52<2:03:55, 1318.86it/s]
  3%|▎         | 303031/10125000.0 [09:52<2:14:11, 1219.87it/s]
  3%|▎         | 271953/10125000.0 [09:52<2:18:31, 1185.45it/s]
  3%|▎         | 259560/10125000.0 [09:52<2:31:26, 1085.70it/s]
  2%|▏         | 233586/10125000.0 [09:53<2:52:50, 953.78it/s]
  3%|▎         | 306936/10125000.0 [09:52<2:02:51, 1331.93it/s]
  3%|▎         | 318801/10125000.0 [09:52<2:04:17, 1314.93it/s]
  3%|▎         | 303810/10125000.0 [09:53<2:15:40, 1206.41it/s]
  3%|▎         | 260281/10125000.0 [09:53<2:33:36, 1070.31it/s]
  3%|▎         | 272691/10125000.0 [09:53<2:26:18, 1122.37it/s]
  3%|▎         | 307720/10125000.0 [09:53<2:01:53, 1342.31it/s]
  2%|▏         | 234270/10125000.0 [09:53<2:50:23, 967.46it/s]
  3%|▎         | 319600/10125000.0 [09:53<2:04:26, 1313.29it/s]
  3%|▎         | 304590/10125000.0 [09:53<2

  3%|▎         | 324415/10125000.0 [10:06<1:59:04, 1371.80it/s]
  3%|▎         | 287661/10125000.0 [10:06<2:24:36, 1133.82it/s]
  2%|▏         | 246051/10125000.0 [10:06<2:49:10, 973.24it/s] 
  3%|▎         | 274170/10125000.0 [10:06<2:27:16, 1114.83it/s]
  3%|▎         | 337431/10125000.0 [10:06<1:54:46, 1421.22it/s]
  3%|▎         | 320400/10125000.0 [10:06<2:08:35, 1270.82it/s]
  3%|▎         | 325221/10125000.0 [10:06<1:58:14, 1381.23it/s]
  3%|▎         | 274911/10125000.0 [10:07<2:26:39, 1119.36it/s]
  3%|▎         | 288420/10125000.0 [10:07<2:22:24, 1151.19it/s]
  3%|▎         | 338253/10125000.0 [10:06<1:56:34, 1399.12it/s]
  2%|▏         | 246753/10125000.0 [10:07<2:46:56, 986.21it/s]
  3%|▎         | 321201/10125000.0 [10:07<2:07:53, 1277.65it/s]
  3%|▎         | 326028/10125000.0 [10:07<1:58:30, 1378.13it/s]
  3%|▎         | 289180/10125000.0 [10:07<2:21:47, 1156.10it/s]
  3%|▎         | 339076/10125000.0 [10:07<1:58:35, 1375.35it/s]
  3%|▎         | 275653/10125000.0 [10:08

  3%|▎         | 289941/10125000.0 [10:20<2:22:08, 1153.23it/s]
  3%|▎         | 341551/10125000.0 [10:20<2:21:05, 1155.71it/s]
  4%|▎         | 356590/10125000.0 [10:20<1:53:40, 1432.14it/s]
  3%|▎         | 338253/10125000.0 [10:20<2:07:04, 1283.62it/s]
  3%|▎         | 304590/10125000.0 [10:21<2:20:58, 1161.07it/s]
  3%|▎         | 261003/10125000.0 [10:21<2:40:45, 1022.61it/s]
  4%|▎         | 357435/10125000.0 [10:20<1:53:30, 1434.28it/s]
  3%|▎         | 342378/10125000.0 [10:21<2:26:58, 1109.39it/s]
  3%|▎         | 339076/10125000.0 [10:21<2:05:54, 1295.38it/s]
  3%|▎         | 290703/10125000.0 [10:21<2:40:34, 1020.70it/s]
  3%|▎         | 305371/10125000.0 [10:21<2:19:06, 1176.44it/s]
  4%|▎         | 358281/10125000.0 [10:21<1:53:12, 1437.84it/s]
  3%|▎         | 261726/10125000.0 [10:22<2:55:37, 936.05it/s] 
  3%|▎         | 343206/10125000.0 [10:21<2:18:07, 1180.27it/s]
  3%|▎         | 339900/10125000.0 [10:21<2:04:46, 1307.06it/s]
  3%|▎         | 306153/10125000.0 [10:2

  4%|▎         | 377146/10125000.0 [10:34<1:44:32, 1554.15it/s]
  4%|▎         | 356590/10125000.0 [10:34<2:01:00, 1345.34it/s]
  3%|▎         | 274170/10125000.0 [10:35<2:44:05, 1000.54it/s]
  4%|▎         | 360825/10125000.0 [10:35<1:53:13, 1437.37it/s]
  3%|▎         | 306153/10125000.0 [10:35<2:16:26, 1199.36it/s]
  3%|▎         | 322003/10125000.0 [10:35<2:05:34, 1301.05it/s]
  4%|▎         | 378015/10125000.0 [10:35<1:44:16, 1558.00it/s]
  4%|▎         | 357435/10125000.0 [10:35<2:00:56, 1346.09it/s]
  4%|▎         | 361675/10125000.0 [10:35<1:51:44, 1456.19it/s]
  3%|▎         | 274911/10125000.0 [10:36<2:46:28, 986.14it/s] 
  3%|▎         | 306936/10125000.0 [10:36<2:22:35, 1147.53it/s]
  4%|▎         | 358281/10125000.0 [10:35<1:58:53, 1369.19it/s]
  4%|▎         | 378885/10125000.0 [10:35<1:56:39, 1392.39it/s]
  3%|▎         | 322806/10125000.0 [10:36<2:22:59, 1142.55it/s]
  4%|▎         | 362526/10125000.0 [10:36<2:01:49, 1335.67it/s]
  3%|▎         | 275653/10125000.0 [10:3

In [ ]:
ray.shutdown()

# Save Results in CSV format 

In [ ]:
# Save data as csv
import glob
import os

path = './results/data'
files = glob.glob(os.path.join(path, "*"+version+".csv"))
display(files)

df = (pd.read_csv(f).assign(DRS = f.split(" - ")[-1].split(version)[0]) for f in files)
df_all_drs = pd.concat(df, ignore_index=True)
df_all_drs.index.name = 'index'
df_all_drs.to_csv('./data/SEAIRD_sigmaOpt_AllDRS'+version+'.csv', sep=",")